In [1]:
import sys
sys.path.append('generation')

In [2]:
from utils import create_generator, progressive_batch_processor

In [3]:
generator = create_generator("openai", "gpt-3.5-turbo")

INFO:openai_generator:Initialized OpenAI generator with model: gpt-3.5-turbo


In [28]:
import pandas as pd

# Load your data
df = pd.read_csv('data/raw/Twitter_Analysis.csv')

In [30]:
filtered_df = df[df['majority_target'] == 0]
print(len(filtered_df))

65213


In [31]:
texts = filtered_df['tweet'].tolist()

In [32]:
generator = create_generator("openai", "gpt-3.5-turbo")

INFO:openai_generator:Initialized OpenAI generator with model: gpt-3.5-turbo


In [33]:
result = progressive_batch_processor(
    generator=generator,
    texts=texts,
    batch_size=10,
    max_items=100,  # Process up to 100 items total
    max_facts=3,
    domain="covid"  # or "general", "news", etc.
)

Processing Batch 1: items 1 to 10
Progress: 0/100 → 10/100
Processing item 1/100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 2/100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 3/100


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 9/100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 10/100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [34]:
print(f"✅ Batch completed: {result['items_processed']} items")
print(f"📊 Total progress: {result['total_processed']}/100")
print(f"🎯 Success rate: {result['quality_metrics'].success_rate:.1%}")

✅ Batch completed: 10 items
📊 Total progress: 10/100
🎯 Success rate: 100.0%
